# QP（二次规划）简介

In [1]:
# Python 库
import numpy as np
from pydrake.all import MathematicalProgram, Solve, eq, ge, le

# MathematicalProgram 简介 

本练习的目的是让你熟悉优化问题实例的基本概念，以及如何求解它。

一个优化问题通常写作：

$$\begin{aligned} \min_x \quad & f(x) \\ \textrm{s.t.} \quad & g(x)\leq 0,\\ \quad &  h(x)=0 \end{aligned}$$

我们称 $x$ 为**决策变量**，$f(x)$ 为**目标函数**，$g(x)\leq 0$ 为**不等式约束**，$h(x)=0$ 为**等式约束**。我们通常用 $x^*$ 表示最优解。大多数情况下，约束是硬约束，意味着最优解必须满足这些约束条件。

Drake 提供了一个非常好的接口 `MathematicalProgram`，可以调用多种求解器。让我们用 `MathematicalProgram` 试着解决一个简单问题：

$$\begin{aligned} \min_x \quad & \frac{1}{2}x^2 \\ \textrm{s.t.} \quad & x\geq 3 \end{aligned}$$

在开始编程之前，你认为答案应该是多少？你应该说服自己，最优解是 $x^*=3$，因为这是在不违反约束的情况下取得最小代价的值。

In [2]:
"""
使用 Drake 的 MathematicalProgram 求解优化问题的步骤
"""

# 1. 定义一个 MathematicalProgram 实例
prog = MathematicalProgram()

# 2. 添加决策变量
x = prog.NewContinuousVariables(1)

# 3. 添加目标函数
prog.AddCost(x.dot(x))

# 4. 添加约束条件
prog.AddConstraint(x[0] >= 3)

# 5. 求解问题
result = Solve(prog)

# 6. 获取解
if result.is_success():
    print("解为: " + str(result.GetSolution()))

解为: [3.]


你应该已经看到我们成功得到了期望的解 $x^*=3$。

我们本节重点关注的一类问题是[二次规划（QP）](https://en.wikipedia.org/wiki/Quadratic_programming)，在实际中可以非常高效地求解（甚至可以达到 kHz 级别）。

这类问题的一般形式如下：

$$\begin{aligned} \min_x \quad & \frac{1}{2}x^T\mathbf{Q}x + c^Tx \\ \textrm{s.t.} \quad & \mathbf{A}x\leq b,\\ \quad &  \mathbf{A}'x=b' \end{aligned}$$

其中 $\mathbf{Q}$ 是正定对称矩阵。注意目标函数是决策变量的二次函数，而约束都是线性的，这就是凸二次规划（QP）的定义。

让我们练习解决一个简单的 QP：

$$\begin{aligned} \min_{x_0,x_1,x_2} \quad & x_0^2 + x_1^2 + x_2^2 \\ \textrm{s.t.} \quad & \begin{pmatrix} 2 & 3 & 1 \\ 5 & 1 & 0 \end{pmatrix} \begin{pmatrix} x_0 \\ x_1 \\ x_2 \end{pmatrix} = \begin{pmatrix} 1 \\ 1 \end{pmatrix}\\  \quad &  \begin{pmatrix} x_0 \\ x_1 \\ x_2 \end{pmatrix} \leq \begin{pmatrix} 2 \\ 2 \\ 2\end{pmatrix} \end{aligned}$$

为了方便地写出向量约束，Drake 提供了 `eq, le, ge` 用于元素级约束。你可能需要一些时间来熟悉这些约束的语法。想要更深入了解 `MathematicalProgram`，[这个 notebook 教程](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/mathematical_program-4c4f4de7d5d3452daacf45c20b315d31) 非常有用。

In [3]:
prog = MathematicalProgram()

x = prog.NewContinuousVariables(3)

prog.AddCost(x.dot(x))
prog.AddConstraint(eq(np.array([[2, 3, 1], [5, 1, 0]]).dot(x), [1, 1]))
prog.AddConstraint(le(x, 2 * np.ones(3)))

result = Solve(prog)

# 6. 获取解
if result.is_success():
    print("解为: " + str(result.GetSolution()))

解为: [0.15897436 0.20512821 0.06666667]



**现在轮到你来解决一个简单问题！** 

你需要解决如下问题，并将结果存储在变量 `result_submission` 中。

$$\begin{aligned} \min_{x_0,x_1,x_2} \quad & 2x_0^2 + x_1^2 + 4x_2^2 \\ \textrm{s.t.} \quad & \begin{pmatrix} 1 & 2 & 3 \\ 2 & 7 & 4 \end{pmatrix} \begin{pmatrix} x_0 \\ x_1  \\ x_2 \end{pmatrix} = \begin{pmatrix} 1 \\ 1 \end{pmatrix} \\ \quad &  |x| \leq \begin{pmatrix} 0.35 \\ 0.35 \\ 0.35\end{pmatrix} \end{aligned}$$

注意：最后一个约束表示 `x[i]` 的绝对值必须小于 `b_bb[i]` 的值。你不能直接将绝对值作为约束，因此有两种方法可以实现：
- 将约束拆分为不含绝对值的两个约束。  
- Drake 提供了 [`AddBoundingBoxConstraint`](https://drake.mit.edu/pydrake/pydrake.solvers.html?highlight=addboundingboxconstraint#pydrake.solvers.MathematicalProgram.AddBoundingBoxConstraint)，你也可以在实现中使用。

In [15]:
prog = MathematicalProgram()

# 添加决策变量
x = prog.NewContinuousVariables(3)

# 添加目标函数：2x0² + x1² + 4x2²
prog.AddCost(2 * x[0]**2 + x[1]**2 + 4 * x[2]**2)

# 添加等式约束：A x = b
A = np.array([[1, 2, 3], [2, 7, 4]])
b = np.array([1, 1])
prog.AddConstraint(eq(A.dot(x), b))

# 添加边界约束：|x| <= 0.35
prog.AddBoundingBoxConstraint(-0.35, 0.35, x)

# 求解问题
result = Solve(prog)

# 获取解并存储
if result.is_success():
    result_submission = result.GetSolution()
else:
    result_submission = None

## 本 notebook 如何评分？

如果你已选课，本 notebook 会通过 [Gradescope](www.gradescope.com) 进行评分。你应该已经在 Piazza 的公告中获得了课程码。

提交本作业时，你需要：
- 下载并提交 `intro_to_qp.ipynb` 到 Gradescope 的 notebook 提交区，并一同提交其他题目的 notebook。

我们会在 notebook 中评测本地函数，检查其行为是否符合预期。本题的评分标准如下：
- [4 分] `result_submission` 必须包含 QP 的正确答案。

下面是我们的自动评分器，你可以用它来检查你的实现是否正确。

In [16]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_simple_qp import TestSimpleQP

Grader.grade_output([TestSimpleQP], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 4/4.

Score for Test Simple QP problem is 4/4.
